# SETUP

In [1]:
import numpy as np
import pandas as pd

from scipy.io import mmwrite, mmread
from konlpy.tag import Komoran
komoran = Komoran()

from collections import Counter
from pprint import pprint

import pickle
import os
import re
import gc

In [2]:
gc.collect()

8

In [3]:
cs_df = pd.read_pickle('cs_whole.pkl')
cs_df.tail()

,date,title,body
1017,2017.03.06,"[사설] 운명의 일주일, '탄핵' '기각' 이후가 더 중요하다",이르면 이번 주 후반 박근혜 대통령 탄핵 심판 사건에 대한 헌재 결정이 나올 가능성...
1018,2017.03.07,[사설] 중국 땅인지 착각게 한 롯데 앞 촛불 시위대,지난 4일 밤 광주 롯데백화점 앞 촛불 시위 사진은 눈을 의심케 했다. 수백 명이 ...
1019,2017.03.07,"[사설] 특검, '최순실 국정농단'의 本流 꿰뚫은 수사였나",박영수 특검팀이 6일 수사 결과를 발표했다. 그러나 평가는 엇갈린다. 구속기소된 이...
1020,2017.03.09,"[사설] 10일 탄핵 심판 선고, 모두 自重하고 또 自制하자",박근혜 대통령 탄핵 심판 선고 기일이 10일로 확정됐다. 헌재 재판관 8명이 8일 ...
1021,2017.03.10,"[사설] 오늘 시험대 오르는 대한민국, '역사적 승복'으로 위기 끝내자",헌법재판소가 오늘 11시 박근혜 대통령 탄핵심판 사건을 선고한다. 작년 10월 5일...


In [4]:
# 상위 기사 100개로 corpuses 만들기
corpuses = []
for doc in list(cs_df.body[:100]):
    corpuses.append(re.sub('\n', '', doc))

### 단어 빈도수 계산

In [5]:
tokens = []
tokens.append([pos for corpus in corpuses for pos in komoran.pos(corpus)])
len(tokens[0])

69311

In [6]:
counter = Counter(tokens[0])
counter = {
    word:freq for word, freq in counter.items()
    if (freq >= 4) and (word[1][:2] == 'NN')
}

pprint(sorted(counter.items(), key=lambda x:-x[1]))

[(('것', 'NNB'), 494),
 (('대통령', 'NNG'), 350),
 (('당', 'NNG'), 347),
 (('선인', 'NNP'), 344),
 (('수', 'NNB'), 253),
 (('박', 'NNP'), 241),
 (('정부', 'NNG'), 216),
 (('국민', 'NNG'), 205),
 (('인사', 'NNG'), 190),
 (('년', 'NNB'), 175),
 (('일', 'NNB'), 169),
 (('북한', 'NNP'), 157),
 (('북', 'NNP'), 146),
 (('핵', 'NNG'), 132),
 (('후보자', 'NNG'), 127),
 (('대선', 'NNG'), 117),
 (('야당', 'NNG'), 114),
 (('박근혜', 'NNP'), 112),
 (('공약', 'NNG'), 112),
 (('청와대', 'NNP'), 111),
 (('국가', 'NNG'), 108),
 (('총리', 'NNG'), 105),
 (('경제', 'NNG'), 102),
 (('때', 'NNG'), 101),
 (('안보', 'NNG'), 99),
 (('원', 'NNB'), 98),
 (('명', 'NNB'), 92),
 (('국회', 'NNG'), 90),
 (('문제', 'NNG'), 89),
 (('사람', 'NNG'), 88),
 (('당선', 'NNG'), 87),
 (('정책', 'NNG'), 85),
 (('말', 'NNG'), 84),
 (('복지', 'NNP'), 83),
 (('장관', 'NNG'), 83),
 (('나라', 'NNG'), 82),
 (('국정', 'NNG'), 82),
 (('정권', 'NNG'), 80),
 (('정치', 'NNG'), 80),
 (('미국', 'NNP'), 79),
 (('부처', 'NNP'), 78),
 (('등', 'NNB'), 76),
 (('데', 'NNB'), 75),
 (('상황', 'NNG'), 70),
 (('후보', 'NNG'), 6

### 사용자 사전 추가

In [7]:
# komoran_userdic = Komoran(userdic='./userdic.txt')
# komoran_userdic.pos(tokens)

### Term Frequency Vector 만들기

In [8]:
# 명사 갯수 세어보기

noun_counter = Counter(
    [noun for corpus in corpuses for noun in komoran.nouns(corpus)]
)

print('count of nouns: %d' % len(noun_counter))

count of nouns: 4032


In [9]:
for min_count in [2, 3, 5, 10]:
    _counter = {
        word for word, freq in noun_counter.items()
        if freq >= min_count
    }
    print('count of nouns (min_count = %d): %d' % (
    min_count, len(_counter)))

count of nouns (min_count = 2): 2178
count of nouns (min_count = 3): 1533
count of nouns (min_count = 5): 994
count of nouns (min_count = 10): 492


In [10]:
# komoran.nouns와 tokenizer 비교하기

noun_dict = {
    word for word, freq in noun_counter.items()
    if freq >= 5
}

def customer_tokenizer(doc):
    return [word for word in komoran.nouns(doc) if word in noun_dict]

In [11]:
# countvectorizer 사용하기

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    tokenizer=customer_tokenizer,
)

x_sparse = vectorizer.fit_transform(corpuses)

In [12]:
type(x_sparse), x_sparse.shape

(scipy.sparse.csr.csr_matrix, (100, 994))

In [13]:
x_dense = x_sparse.todense()
print(x_dense.shape)
x_dense

(100, 994)


matrix([[0, 0, 0, ..., 3, 0, 0],
        [0, 0, 0, ..., 2, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

### countervectorizer에서 각 column에 해당하는 term 알아내기

In [14]:
vocab2int = vectorizer.vocabulary_
vocab2int['박근혜']

309

In [15]:
int2vocab = [
    word for word,index in sorted(
        vocab2int.items(), key=lambda x:x[1])
]
int2vocab[20]

'감시'

### Sparse matrix I/O

In [16]:
if not os.path.exists('tmp/'):
    os.makedirs('tmp')
    
mmwrite('./tmp/x.mm', x_sparse)

In [17]:
loaded_x_sparse = mmread('./tmp/x.mm')
loaded_x_sparse = loaded_x_sparse.tocsr()
loaded_x_sparse

<100x994 sparse matrix of type '<class 'numpy.int64'>'
	with 10341 stored elements in Compressed Sparse Row format>

### DataFrame 만들기

In [18]:
words = vectorizer.vocabulary_.keys()
docs = list(cs_df.title[:100])

In [19]:
cv_noun_df = pd.DataFrame(data=x_dense, index=docs, columns=words)
cv_noun_df.tail()

,일,실시,대통령,선거,새누리당,박근혜,후보,명,중,표,...,정전,협정,검사,화법,석,백년 전쟁,팀원,특별감찰관,상설,특검
[사설] 현대史를 '총칼 없는 백년 전쟁'으로 몰아가는 좌파,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
[사설] 국세청 '뇌물 잔치' 보며 국민이 세금 낼 기분 나겠나,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
"[사설] 대통령, 이제 人事에 관한 국민 感想 들어볼 때다",0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,1,0,0,0,0,0
"[사설] 정부조직법 대치 47일, 정치는 없었다",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"[사설] 특별감찰관·상설특검제, 해결해야 할 과제 많다",0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


### Save vectorizer and vocabulary

In [20]:
with open('./tmp/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
    
with open('./tmp/vectorizer.txt', 'w', encoding='utf-8') as f:
    for vocab in int2vocab:
        f.write('%s\n' % vocab)

In [21]:
with open('./tmp/vectorizer.pkl', 'rb') as f:
    loaded_vectorizer = pickle.load(f)
    
with open('./tmp/vectorizer.txt', 'r', encoding='utf-8') as f:
    for _ in range(10):
        print(next(f).strip())

10년
2월
2월 25일
5년
가계
가구
가량
가운데
가입
가족


### build bigram dictionary

In [22]:
len(corpuses)

100

In [23]:
def bigram_extractor(docs, min_count=10):

    def to_bigram(tokens):
        bigrams = [(t0, t1) for t0, t1 in zip(tokens, tokens[1:])]
        return bigrams

    bigram_counter = Counter(
        [bigram for doc in docs 
         for bigram in to_bigram(komoran.pos(doc, join=True)) if doc]
    )

    bigram_dictionary = {
        bigram:count for bigram, count in bigram_counter.items() if count >= min_count
    }
    
    return bigram_dictionary

bigrams = bigram_extractor(corpuses)
len(bigrams)

692

In [24]:
list(bigrams)

[('되/XSV', 'ㄴ/ETM'),
 ('후보/NNG', '가/JKS'),
 ('만/NR', '명/NNB'),
 ('되/XSV', '었/EP'),
 ('었/EP', '다/EF'),
 ('다/EF', './SF'),
 ('후보/NNG', '는/JX'),
 ('%/SW', '를/JKO'),
 ('대통령/NNG', '은/JX'),
 ('이/VCP', '다/EF'),
 ('을/ETM', '것/NNB'),
 ('것/NNB', '으로/JKB'),
 ('하/XSV', '았/EP'),
 ('았/EP', '던/ETM'),
 ('았/EP', '다/EF'),
 ('대통령/NNG', '이/JKS'),
 ('./SF', '박/NNP'),
 ('야당/NNG', '이/JKS'),
 ('국민/NNG', '의/JKG'),
 ('이/JKS', '더/MAG'),
 ('어/EC', '지/VX'),
 ('이/VCP', '었/EP'),
 ('크/VA', '게/EC'),
 ('에서/JKB', 'ㄴ/JX'),
 ('에게/JKB', 'ㄴ/JX'),
 ('%/SW', '포인트/NNP'),
 ('전/NNG', '에/JKB'),
 ('에/JKB', '는/JX'),
 ('기/ETN', '도/JX'),
 ('하/VV', '았/EP'),
 ('박/NNP', '당/NNG'),
 ('당/NNG', '선인/NNP'),
 ('선인/NNP', '에게/JKB'),
 ('하/XSA', 'ㄴ/ETM'),
 ('하/XSV', 'ㄴ/ETM'),
 ('과/JC', '함께/MAG'),
 ('자신/NNG', '의/JKG'),
 ('그/NP', '들/XSN'),
 ('들/XSN', '을/JKO'),
 ('는/ETM', '것/NNB'),
 ('것/NNB', '이/VCP'),
 ('5/SN', '년/NNB'),
 ('년/NNB', '동안/NNG'),
 ('속/NNG', '에서/JKB'),
 ('에서/JKB', '도/JX'),
 ('되/XSV', '고/EC'),
 ('./SF', '그렇/VA'),
 ('에/JKB', '도/JX'),
 ('국민

In [25]:
# bigram tokenizer

class BigramTokenizer:
    
    def __init__(self, bigrams, tagger):
        self.bigrams = bigrams
        self.tagger = tagger
        
    def __call__(self, sent):
        if not sent:
            return []
        
        unigrams = self.tagger.pos(sent, join=True)
        
        bigrams = [(t0, t1) for t0, t1 in zip(unigrams, unigrams[1:])]
        bigrams = [bigram for bigram in bigrams if bigram in self.bigrams]
        bigrams = ['%s-%s' % (t0, t1) for t0, t1 in bigrams]
        
        return unigrams + bigrams

In [26]:
bigram_tokenizer = BigramTokenizer(bigrams, komoran)

In [27]:
sample_1 = corpuses[0]
sample_1[:200]

'19일 실시된 대통령선거에서 새누리당 박근혜 후보가 투표자 3072만명 중 51% 남짓 1600만표가량을 얻어 18대 대통령에 당선됐다. 민주통합당 문재인 후보는 48% 남짓 1500만표가량을 얻었다. 1987년 직선제 개헌 이후 득표율 50%를 넘긴 대통령은 이번이 처음이다. 70%를 약간 넘을 것으로 예상했던 투표율은 75.8%까지 올랐다. 대한민국 건'

In [28]:
bigram_1 = bigram_tokenizer(sample_1)
len(bigram_1)

2310

### cohesion_ltokenizer

In [39]:
from collections import defaultdict
import sys
sys.path.append('../yourpackage/')

from corpus import DoublespaceLineCorpus

In [40]:
def get_cohesion(word):
    
    # 글자가 아니거나 공백, 혹은 희귀한 단어인 경우
    if (not word) or ((word in L) == False):
        return 0.0
    
    n = len(word)
    if n == 1:
        return 0
    
    word_freq = L.get(word, 0)
    base_freq = L.get(word[:1], 0)
    
    if base_freq == 0:
        return 0.0
    else:
        return pow((word_freq / base_freq), 1 / (n - 1))

In [41]:
for e in range(1, 5+1):
    print('abcdefg'[:e])

a
ab
abc
abcd
abcde


In [42]:
L = defaultdict(int)
for corpus in corpuses[:10]:
    for eojeol in corpus.split():
        for e in range(1, len(eojeol)+1):
            subword = eojeol[:e]
            L[subword] += 1
print('num subword = %d' % len(L))

num subword = 5529


In [43]:
w = '박근혜가'
for e in range(1, 5):
    print(w[:e], get_cohesion(w[:e]))

박 0
박근 0.2692307692307692
박근혜 0.5188745216627708
박근혜가 0.0


In [48]:
cohesion_score = {}
for word, count in L.items():
    if count < 10 or len(word) < 2:
        continue
    cohesion_score[word] = get_cohesion(word)
    
print('n computed = {}'.format(len(cohesion_score)))

n computed = 54
